In [1]:
import sys
sys.path.append('../')
from rocky.core import rocky
from rocky.triangle import Triangle
from rocky.model_selection.TriangleTimeSeriesSplit import TriangleTimeSeriesSplit
import pandas as pd
import numpy as np

In [2]:
# Create a Rocky object
rky = rocky()

# add triangle
rocky_inputs_file = r"O:\PARM\Individual folders\A Weaver\Reserving\rocky\wc-icrfs.xlsx"
rky.FromExcel(filename=rocky_inputs_file, sheet_name='triangle', id='paid_loss')
# rky.SampleTri('taylor_ashe')

# add model
rky.AddModel('icrfs', model_class='loglinear', tri=rky.paid_loss, use_cal=True, plot_width=None)

column:
0

cal[c]:
accident_period
1967-01-01     0
1968-01-01     1
1969-01-01     2
1970-01-01     3
1971-01-01     4
1972-01-01     5
1973-01-01     6
1974-01-01     7
1975-01-01     8
1976-01-01     9
1977-01-01    10
1978-01-01    11
1979-01-01    12
1980-01-01    13
1981-01-01    14
1982-01-01    15
1983-01-01    16
1984-01-01    17
1985-01-01    18
1986-01-01    19
1987-01-01    20
1988-01-01    21
1989-01-01    22
1990-01-01    23
1991-01-01    24
1992-01-01    25
1993-01-01    26
1994-01-01    27
1995-01-01    28
1996-01-01    29
1997-01-01    30
1998-01-01    31
1999-01-01    32
2000-01-01    33
2001-01-01    34
2002-01-01    35
2003-01-01    36
2004-01-01    37
2005-01-01    38
2006-01-01    39
2007-01-01    40
2008-01-01    41
2009-01-01    42
2010-01-01    43
2011-01-01    44
2012-01-01    45
2013-01-01    46
2014-01-01    47
2015-01-01    48
2016-01-01    49
2017-01-01    50
2018-01-01    51
2019-01-01    52
2020-01-01    53
2021-01-01    54
2022-01-01    55
2023-01-01   

ValueError: cannot convert float NaN to integer

In [4]:
# set hyperparameters
rky.icrfs.SetHyperparameters(alpha=0.05, l1_ratio=0.01)
# rky.icrfs.TuneFitHyperparameters(l1_ratio=[0, 0.025, 0.05])

# fit model
rky.icrfs.Fit()

# basic hetero adjustment (one-step, no cross-validation,
# just one parameter per development period)
rky.icrfs.FitHetero()

Fitting hetero adjustment: (Step/RMSE/L2-Norm)
1/0.0000 

In [5]:
rky.icrfs.GetIdx()

0         0
1         1
2         2
3         3
4         4
       ... 
734    1331
735    1332
736    1368
737    1369
738    1406
Length: 739, dtype: int64

In [6]:
rky.icrfs.GetCVStatistics()

KeyError: '[924, 925] not in index'

^^ this doesn't work when using a real triangle?

In [ ]:
rky.icrfs.LassoFeatureImportance()

,0.0000,0.0025,0.0050,0.0075,0.0100,0.0125,0.0150,0.0175,0.0200,0.0225,...,0.2600,0.2625,0.2650,0.2675,0.2700,0.2725,0.2750,0.2775,0.2800,0.2825
0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,105.0,106.0,107.0,108.0,109.0,110.0,111.0,112.0,113.0,114.0
1,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,105.0,106.0,107.0,108.0,109.0,110.0,111.0,112.0,113.0,114.0
2,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,105.0,106.0,107.0,108.0,109.0,110.0,111.0,112.0,113.0,114.0
3,114.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,104.0,105.0,106.0,107.0,108.0,109.0,110.0,111.0,112.0,113.0
4,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,105.0,106.0,107.0,108.0,109.0,110.0,111.0,112.0,113.0,114.0
5,2.0,1.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,105.0,106.0,107.0,108.0,109.0,110.0,111.0,112.0,113.0,114.0
6,2.0,1.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,105.0,106.0,107.0,108.0,109.0,110.0,111.0,112.0,113.0,114.0
7,2.0,1.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,105.0,106.0,107.0,108.0,109.0,110.0,111.0,112.0,113.0,114.0
8,114.0,113.0,112.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,...,102.0,103.0,104.0,105.0,106.0,107.0,108.0,109.0,110.0,111.0
9,114.0,113.0,112.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,...,102.0,103.0,104.0,105.0,106.0,107.0,108.0,109.0,110.0,111.0


In [ ]:
rky.icrfs.tri.ult()

Accident Period
2001-01-01    3901463.0
2002-01-01    5433719.0
2003-01-01    5378826.0
2004-01-01    5297906.0
2005-01-01    4858200.0
2006-01-01    5111171.0
2007-01-01    5660771.0
2008-01-01    6784799.0
2009-01-01    5642266.0
2010-01-01    4969825.0
Name: Chain Ladder Ultimate Loss, dtype: float64

In [ ]:
rky.icrfs.Ultimate()

accident_period
2001-01-01    2567142.0
2002-01-01    3706315.0
2003-01-01    3638545.0
2004-01-01    3629865.0
2005-01-01    3506981.0
2006-01-01    3807868.0
2007-01-01    4326249.0
2008-01-01    4866519.0
2009-01-01    4457979.0
2010-01-01    4296988.0
Name: Log-linear Ultimate, dtype: float64

In [ ]:
rky.icrfs.TrendPlot()

In [ ]:
#test 
rky.icrfs.FitPlot('acc')

In [ ]:
rky.icrfs.ResidualPlot(color_col='acc')

$\text{AIC} = 2k - 2l$

In [ ]:
# k - number of parameters:
rky.icrfs.GetP()

37

In [ ]:
# l - loglikelihood:
# log(likelihood)


In general

Objective function of elastic net regularization:

$$
\min_{\beta} \frac{1}{2n} \sum_{i=1}^n \left( y_i - \beta_0 - \mathbf{x}_i^T\beta \right)^2 + \alpha \lambda \|{\beta}\|^1 + \frac{\alpha(1-\lambda)}{2} \|\beta\|_2^2
$$

Here $\alpha$ is the strength of the regularization, $\lambda$ is the portion of the regularization that is $\ell_1$-norm, and $k$ is the number of parameters in the model.

